In [38]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_excel('transactiondata.xlsx')
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
df.columns

In [ ]:
# Family spending habits
familydf = df[['Family ID','Income', 'Savings', 'Monthly Expenses', 'Loan Payments','Credit Card Spending', 'Dependents', 'Financial Goals Met (%)']]
familydf = familydf.drop_duplicates()
familydf

In [ ]:
# Member specific spending habits
memberdf = df[['Family ID', 'Category', 'Amount']]
vis1 = df[['Category', 'Amount']]
memberdf

In [ ]:
# Let us explore the member specific spending habits and the see what we can learn
memberdf['Category'].unique()

In [ ]:
# Perhaps if we combine all the expenses and split them into lavish and neccesary expenditures
memberdf.loc[memberdf['Category'].isin(['Travel', 'Food', 'Entertainment']),'Type'] = 'Lavish'
memberdf.loc[memberdf['Category'].isin(['Groceries','Healthcare','Education','Utilities']), 'Type'] = 'Necessity'
memberdf

In [ ]:
memberdf = memberdf[['Family ID', 'Amount', 'Type']].groupby(['Family ID', 'Type']).sum().reset_index()
# memberdf['Lavish to Necessity %'] = (memberdf['Amount'][memberdf['Type'] == 'Lavish'] / memberdf['Amount'][memberdf['Type'] == 'Necessity']) * 100
memberdf


In [ ]:
memberdf = memberdf[['Family ID', 'Amount', 'Type']].pivot(index='Family ID', columns='Type', values='Amount')
memberdf['Lavish to Necessity %'] = (memberdf['Lavish'] / memberdf['Necessity']) * 100
memberdf = memberdf.reset_index()
memberdf

In [ ]:
#Now moving on to familydf
familydf.head()

In [ ]:
familydf.describe(include='all')

In [ ]:
familydf.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
family_corr = familydf[['Income', 'Savings', 'Monthly Expenses', 'Loan Payments','Credit Card Spending', 'Dependents', 'Financial Goals Met (%)']].corr()

sns.heatmap(family_corr, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Income vs expense correlation is -0.069
# Saving vs Credit Card spending correlation is 0.0058 only
sns.pairplot(familydf)
plt.show()

In [ ]:
# No genuine correlations. Perhaps some feature engineering
# Building Financial scoring model factors simultaneously as instructed
familydf['Savings to Income'] = familydf['Savings']/familydf['Income']
familydf['Expense Percentage'] = (familydf['Monthly Expenses']/familydf['Income'])*100
familydf['Loan Percentage'] = (familydf['Loan Payments']/familydf['Income'])*100
familydf['Credit Utilization Percentage'] = (familydf['Credit Card Spending']/familydf['Income'])*100
familydf = familydf.merge(memberdf, how='right',on='Family ID')
familydf

In [ ]:
familydf.columns

In [ ]:
familydf = familydf.drop(['Lavish','Necessity'], axis='columns')

In [ ]:
familydf.describe(include='all')

In [ ]:
corr_matrix = familydf[['Savings to Income', 'Expense Percentage', 'Loan Percentage', 'Credit Utilization Percentage','Financial Goals Met (%)','Lavish to Necessity %']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap of Financial Indicators')
plt.show()


In [ ]:
# STILL no meaningful relationships. 
# There is some moderate linear relationship between credit card vs savings ratio and credit card vs expense ratio.
# We already saw some correlation (as is obvious) between savings and loan amount

In [ ]:
plt.figure(figsize=(15, 10))
familydf[['Savings to Income', 'Expense Percentage', 'Loan Percentage', 'Credit Utilization Percentage','Lavish to Necessity %']].hist(bins=20, edgecolor='black', figsize=(15, 10))
plt.tight_layout()
plt.show()


In [ ]:
# All skewed left, Very few people have more savings than is their income; Most people have only 2-5% of their income as expenses. 
# There is definitely more to learn by utilizing scikit learn as in non-linear relationships (unfortunately, not equipped to handle those yet)
# Now we must try to build a scoring model using all the information we've learned about the dataset

In [ ]:
# Firstly, lets normalize all the scores we will use for building the scoring schema
def normalize(record):
    return (record - record.min())/(record.max() - record.min())*100

familydf['Savings to Income'] = normalize(familydf['Savings to Income'])
familydf['Expense Percentage'] = normalize(familydf['Expense Percentage'])
familydf['Loan Percentage'] = normalize(familydf['Loan Percentage'])
familydf['Credit Utilization Percentage'] = normalize(familydf['Credit Utilization Percentage'])
familydf['Lavish to Necessity %'] = normalize(familydf['Lavish to Necessity %'])
familydf

In [ ]:
# Now I'll have to apply weights to each of these based on how much they potentially impact the financial health of a family
familydf.columns

In [ ]:
familydf[['Financial Goals Met (%)','Savings to Income', 'Expense Percentage', 'Loan Percentage','Credit Utilization Percentage', 'Lavish to Necessity %']].corr()

In [ ]:
# Trying a linear model first to see if we can get acceptable 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

X = familydf[['Savings to Income', 'Expense Percentage', 'Loan Percentage','Credit Utilization Percentage', 'Lavish to Necessity %']]
y = familydf['Financial Goals Met (%)']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=20)

scoremodel = RandomForestRegressor(random_state=20)
scoremodel.fit(X_train, y_train)
y_predicted = scoremodel.predict(X_test)
mse = mean_squared_error(y_test, y_predicted)
print(mse)

# MSE is obviously too high ~20% points


In [ ]:
# For the purposes of this assignment, due to my knowledge and time constraints, i will unfortunately have to use static weights to calculate the financial score. 
# I'll have to compare how much impact each of the features have made with the 'Financial Goals met'
# I'll use (Corr Score/Total Corr Score) in % here
# Total corr score = 0.434; Feature scores respectively: 8%, 22%, 12%, 33%, 25% (rounded off)
weights = {
    'savingstoincomescore': 0.08,
    'expensepercentagescore': 0.22,
    'loanpercentagescore': 0.12,
    'creditutilizationscore': 0.33,
    'lifestylescore': 0.25
}

In [ ]:
familydf['Financial Score'] = (
    familydf['Savings to Income'] * weights['savingstoincomescore'] + familydf['Expense Percentage'] * weights['expensepercentagescore'] + familydf['Loan Percentage'] * weights['loanpercentagescore'] + familydf['Credit Utilization Percentage'] * weights['creditutilizationscore'] + familydf['Lavish to Necessity %'] * weights['lifestylescore']
)
familydf.head()

In [ ]:
familydf.describe(include='all')

In [ ]:
sns.histplot(familydf['Financial Score'], bins=20,)
plt.show()

In [ ]:
# Now to add the recommendations based on their scores
def recommendations(row):
    
    recommendations = []
    # less than the median (50% in .describe())
    if row['Savings to Income'] < 18:
        recommendations += ['Try saving more of your money wrt income']
    elif row['Expense Percentage'] < 17:
        recommendations += ['Attempt reducing your expenses']
    elif row['Loan Percentage'] < 20:
        recommendations += ['Perhaps try reducing your debt burden']
    elif row['Credit Utilization Percentage'] < 20:
        recommendations += ['Try cutting back on credit card purchases']
    elif row['Lavish to Necessity %'] < 47:
        recommendations += ['Try being more mindful of your purchase requirements']

    return recommendations

familydf['Recommendations'] = familydf.apply(recommendations, axis=1)
familydf

In [ ]:
# Visualizations as instructed
# Spending distribution across categories.
vis1 =  vis1.groupby(['Category']).sum()
vis1 = vis1.reset_index()
vis1
plt.figure(figsize=(12,9))
plt.bar(vis1['Category'], vis1['Amount'])
plt.show()

In [ ]:
# Family wise financial score
plt.figure(figsize=(12,9))
plt.bar(familydf['Family ID'],familydf['Financial Score'])
plt.show()

In [ ]:
# Histogram of financial scores
plt.figure(figsize=(12, 9))
sns.histplot(familydf['Financial Score'], bins=20, kde=True)
plt.show()

In [ ]:
# 